In [1]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split # Functipn to split data into training, validation and test sets
from sklearn.metrics import classification_report, confusion_matrix
import pickle
import glob   # The glob module finds all the pathnames matching a specified pattern according to the rules used by the Unix shell, although results are returned in arbitrary order. No tilde expansion is done, but *, ?, and character ranges expressed with [] will be correctly matched.
import os   # miscellneous operating system interfaces. This module provides a portable way of using operating system dependent functionality. If you just want to read or write a file see open(), if you want to manipulate paths, see the os.path module, and if you want to read all the lines in all the files on the command line see the fileinput module.
import random       
from tqdm import tqdm 
from tqdm.notebook import tqdm_notebook
import datetime
import time
from tabulate import tabulate

# Torch
import torch
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
import torchinfo


from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_curve,log_loss
from sklearn.metrics import average_precision_score,roc_auc_score
import os
import time
from time import time
import datetime
import pandas as pd
import numpy as np
import skmultilearn
#from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from skmultilearn.adapt import MLkNN

# CMAP (extracting relevant transcriptomic profiles)
from cmapPy.pandasGEXpress.parse import parse
import cmapPy.pandasGEXpress.subset_gctoo as sg
import seaborn as sns
import matplotlib.pyplot as plt

import datetime
import time as time


In [57]:
# ----------------------------------------- hyperparameters ---------------------------------------#
# Hyperparameters
#testing = False # decides if we take a subset of the data
max_epochs = 75 # number of epochs we are going to run 
# apply_class_weights = True # weight the classes based on number of compounds
using_cuda = True # to use available GPUs
world_size = torch.cuda.device_count()
#---------------------------------------------------------------------------------------------------#
start = time.time()
now = datetime.datetime.now()
now = now.strftime("%d_%m_%Y-%H:%M:%S")

# creating Functional Dataset and Data Loader
1. Downloading all relevant data frames and csv files
2. Parse full gctx file
3. extract the correct transcriptomic profiles
4. prepare train vs test columns using one hot encoding
5. create dataset and data loader

## 1. Downloading all relevant data frames and csv files 

In [58]:
# clue column metadata with columns representing compounds in common with SPECs 1 & 2
clue_sig_in_SPECS = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/init_data_expl/clue_sig_in_SPECS1&2.csv', delimiter = ",")

In [59]:
# clue row metadata with rows representing transcription levels of specific genes
clue_gene = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/init_data_expl/clue_geneinfo_beta.txt', delimiter = "\t")

In [60]:
# the training set, what transcriptomic profiles/compounds are included in only the training set
L1000_training = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/L1000_training_set.csv', delimiter = ",")

In [168]:
# the training set, what transcriptomic profiles/compounds are included in only the training set
L1000_validation = pd.read_csv('/home/jovyan/Tomics-CP-Chem-MoA/04_Tomics_Models/data_split_csvs/L1000_valid_set.csv', delimiter = ",")

In [62]:
num_classes = len(L1000_training['moa'].unique())
num_classes

10

# 2. parsing full gctx file

In [43]:
# preparing the gctoo dataframe to extract from
# choosing only landmark genes
def tprofiles_gc_too_func(L1000_data, clue_gene):
    '''
    
    Input:
    L1000: column meta data from clue.io that only includes training/test data
    clue_gene: row meta data from clue.io transcriptomic profiles
    
    Output:
    parsed gctoo file with all of the transcriptomic profiles'''

    clue_gene["gene_id"] = clue_gene["gene_id"].astype(str)
    landmark_gene_row_ids = clue_gene["gene_id"][clue_gene["feature_space"] == "landmark"]

    # get all samples (across all cell types, doses, and other treatment conditions) with certain MoA
    profile_ids = L1000_data["sig_id"]
    tprofiles_gctoo = parse("/scratch2-shared/erikep/level5_beta_trt_cp_n720216x12328.gctx", 
                                    cid= profile_ids, 
                                    rid = landmark_gene_row_ids)

    ### 2. copy the metadata from those samples
    # first, we need to subset all the metadata information from our larger metadata DataFrame
    # col
    tprofiles_sig_id_info = L1000_training

    # row 
    tprofiles_gene_id_info = clue_gene[clue_gene["feature_space"] == "landmark"]

    tprofiles_sig_id_info.set_index("sig_id", inplace=True)
    # now the data frame is indexed by sig_ids consistent with those of the data_df:
    tprofiles_sig_id_info.index

    tprofiles_gene_id_info.set_index("gene_id", inplace=True)
    # now the data frame is indexed by sig_ids consistent with those of the data_df:
    tprofiles_gene_id_info.index

    # set the relevant annotations as a col_metadata_df: 
    tprofiles_gctoo.col_metadata_df = tprofiles_sig_id_info

    # set the relevant annotations as a row_metadata_df: 
    tprofiles_gctoo.row_metadata_df = tprofiles_gene_id_info
    
    return tprofiles_gctoo

In [44]:
profiles_gc_too = tprofiles_gc_too_func(L1000_training, clue_gene)

### 3.  Extracting the correct transcriptomic profile

In [111]:
def extract_tprofile(profiles_gc_too, idx):
    '''returns transcriptomic profile of of specific ID with in the form of a torch tensor
    Input: 
    index number of row corresponding to specific transcriptomic profile

    Output: 
    tensor of values representing up and down regulated landmark genes

    '''
    # extract unique column name from L1000 data col metadata
    # use it to parse the gctoo file
    # return transcriptomic profile as tensor
    
    tprofile_id =  profiles_gc_too.col_metadata_df.iloc[idx]
    tprofile_id_sig = [tprofile_id.name] 
    tprofile_gctoo = sg.subset_gctoo(profiles_gc_too, cid= tprofile_id_sig) 
    return torch.tensor(tprofile_gctoo.data_df.values.astype(np.float32))     

In [45]:
profiles_gc_too.data_df.shape()

(978, 14115)

### 4. One hot encoding

In [66]:
def splitting(df):
    '''Splitting data into two parts:
    1. input : the pointer showing where the transcriptomic profile is  
    2. target one hot : labels (the correct MoA) '''
    
    # one-hot encoding labels
     # creating tensor from all_data.df
    target = torch.tensor(df['moa'].values.astype(np.int64))

    # For each row, take the index of the target label
    # (which coincides with the score in our case) and use it as the column index to set the value 1.0.” 
    target_onehot = torch.zeros(target.shape[0], num_classes)
    target_onehot.scatter_(1, target.unsqueeze(1), 1.0)
    
    input =  df.drop('moa', axis = 1)
    
    return input, target_onehot


In [67]:
input_df, labels = splitting(L1000_training) 

In [72]:
labels.shape

torch.Size([14115, 10])

In [170]:
x = extract_tprofile(0)
x

tensor([[-1.7367e+00],
        [ 2.8955e-01],
        [-3.6159e-01],
        [-2.0479e-02],
        [-8.9499e-02],
        [ 6.8754e-01],
        [-5.5421e-01],
        [ 1.2909e+00],
        [ 4.6671e-02],
        [-1.6197e+00],
        [ 1.6728e+00],
        [-9.9849e-01],
        [ 6.4678e-01],
        [-8.1987e-01],
        [-3.9679e-01],
        [-1.8376e-01],
        [-8.5614e-01],
        [-1.6580e-01],
        [ 1.6929e+00],
        [-1.9236e-01],
        [ 6.9798e-01],
        [ 1.6317e+00],
        [ 3.2658e-02],
        [ 1.9597e-01],
        [ 2.8483e-01],
        [-3.7714e-01],
        [ 1.4368e+00],
        [-1.1856e+00],
        [ 7.2535e-01],
        [-6.1239e-01],
        [ 2.3079e-01],
        [-2.1131e-01],
        [ 4.1226e-01],
        [-6.7675e-01],
        [-8.1892e-01],
        [-4.3428e-01],
        [ 2.1288e-01],
        [ 1.4696e+00],
        [-1.7285e-01],
        [-1.0520e+00],
        [ 4.6449e-01],
        [-6.3560e-01],
        [ 5.6596e-01],
        [ 3

## 5. Create Data Set

In [158]:
class Transcriptomic_Profiles(torch.utils.data.Dataset):
    def __init__(self, profiles_gc_too, labels):
        self.tprofile_labels = labels
        self.profiles_gc_too = profiles_gc_too

    def __len__(self):
        ''' The number of data points '''
        return len(self.tprofile_labels)

    def __getitem__(self, idx):
        '''Retreiving the transcriptomic profile and label'''
        # print(idx)
        t_profile = extract_tprofile(self.profiles_gc_too, idx) # extract image from csv using index
        # x = input.iloc[idx]
        #print(f' return from function: {image}')
        label = self.tprofile_labels[idx]          # extract calssification using index
        # print(label)
        #print(label)
        #label = torch.tensor(label, dtype=torch.short)
        return t_profile.float(), label.float() # t_profile.float()


In [159]:
batch_size = 1 
# parameters
params = {'batch_size' : 1,
         'num_workers' : 3,
         'shuffle' : True,
         'prefetch_factor' : 2} 
          
if using_cuda:
    device = (torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu'))
else:
    device = torch.device('cpu')
print(f'Training on device {device}. ' )

Training on device cuda. 


In [160]:
# generator: training
# create a subset with only train indices

# create generator that randomly takes indices from the training set
training_dataset = Transcriptomic_Profiles(labels)

In [161]:
training_generator = torch.utils.data.DataLoader(training_dataset, **params)

In [162]:
train_profile, train_labels = next(iter(training_generator))

In [163]:
print(f'profile: {train_profile}, train label: {train_labels}')

profile: tensor([[[ 2.8910e-01],
         [ 4.0230e-01],
         [ 5.4270e-01],
         [ 6.7450e-01],
         [-4.6980e-01],
         [-2.7790e-01],
         [ 4.5860e-01],
         [-9.5200e-01],
         [ 3.8860e-01],
         [-3.5130e-01],
         [-2.8160e-01],
         [ 1.0282e+00],
         [-1.1450e-01],
         [ 2.8310e-01],
         [ 2.7820e-01],
         [ 4.6470e-01],
         [ 1.3060e-01],
         [ 1.4448e+00],
         [ 8.4200e-02],
         [-1.2340e+00],
         [-7.7450e-01],
         [ 9.0170e-01],
         [ 2.2100e-02],
         [ 1.2450e-01],
         [-6.0550e-01],
         [ 3.2910e-01],
         [ 2.2610e-01],
         [-1.1749e+00],
         [-2.7670e-01],
         [ 4.0380e-01],
         [ 3.1120e-01],
         [ 2.2740e-01],
         [ 5.4220e-01],
         [-1.6807e+00],
         [ 1.3636e+00],
         [ 4.3250e-01],
         [-7.1450e-01],
         [ 3.5250e-01],
         [ 7.3040e-01],
         [ 4.9200e-01],
         [-2.4940e-01],
       

We have a functional dataset and data loader

In [3]:
moas = ["cyclooxygenase inhibitor", "dopamine receptor antagonist","adrenergic receptor antagonist", "phosphodiesterase inhibitor",  "HDAC inhibitor", 
             "histamine receptor antagonist","EGFR inhibitor", "adrenergic receptor agonist", "PARP inhibitor",  "topoisomerase inhibitor"]
dictionary = {}
for i,j in enumerate(moas):
    print(i)
    print(type(i))
    print(j)
    dictionary[j] = i
dictionary
    

0
<class 'int'>
cyclooxygenase inhibitor
1
<class 'int'>
dopamine receptor antagonist
2
<class 'int'>
adrenergic receptor antagonist
3
<class 'int'>
phosphodiesterase inhibitor
4
<class 'int'>
HDAC inhibitor
5
<class 'int'>
histamine receptor antagonist
6
<class 'int'>
EGFR inhibitor
7
<class 'int'>
adrenergic receptor agonist
8
<class 'int'>
PARP inhibitor
9
<class 'int'>
topoisomerase inhibitor


{'cyclooxygenase inhibitor': 0,
 'dopamine receptor antagonist': 1,
 'adrenergic receptor antagonist': 2,
 'phosphodiesterase inhibitor': 3,
 'HDAC inhibitor': 4,
 'histamine receptor antagonist': 5,
 'EGFR inhibitor': 6,
 'adrenergic receptor agonist': 7,
 'PARP inhibitor': 8,
 'topoisomerase inhibitor': 9}